
## **IoTSecureScan: An OWASP-Based Security Assessment Tool for IoT Devices**



In [ ]:
!apt-get install binwalk nmap
!pip install python-nmap termcolor requests

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  arj cramfsswap fonts-dejavu-core fonts-lyx freeglut3 libafflib0v5 libbfio1 libdate-manip-perl
  libevdev2 libewf2 libglu1-mesa libgudev-1.0-0 libimagequant0 libiniparser1 libinput-bin
  libinput10 liblbfgsb0 liblinear4 liblua5.3-0 liblzo2-2 libmd4c0 libmtdev1 libpcap0.8 libqt5core5a
  libqt5dbus5 libqt5designer5 libqt5gui5 libqt5help5 libqt5network5 libqt5opengl5
  libqt5printsupport5 libqt5sql5 libqt5sql5-sqlite libqt5svg5 libqt5test5 libqt5widgets5 libqt5xml5
  libraqm0 libtsk19 libvhdi1 libvmdk1 libwacom-bin libwacom-common libwacom9 libxcb-icccm4
  libxcb-image0 libxcb-keysyms1 libxcb-render-util0 libxcb-util1 libxcb-xinerama0 libxcb-xinput0
  libxcb-xkb1 libxkbcommon-x11-0 libxsimd-dev lua-lpeg mtd-utils ncompress nmap-common
  pyqt5-dev-tools python-matplotlib-data python3-appdirs python3-attr python3-beniget
  python3-binwalk pyt

In [ ]:
!pip install fpdf
!pip install pyfiglet

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=643758f975f72d2519265dea95acac7626fc5c2d9e10e678721a9383b2b61ac6
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.1 MB/s eta 0:00:00


In [ ]:
import os
import json
import nmap
import requests
import re
from fpdf import FPDF
import pandas as pd
from termcolor import cprint
from google.colab import files
import pyfiglet
import subprocess

TOOL_NAME = "IoTSecureScan"

# Display Tool Title
def display_title():
    print("=" * 80)
    ascii_art = pyfiglet.figlet_format(TOOL_NAME)
    cprint(ascii_art, "cyan")
    print("=" * 80)

def display_menu():
    display_title()
    cprint("1. Perform SAST Analysis (Firmware Scan)", "green")
    cprint("2. Perform DAST Analysis (Network/Device Scan)", "green")
    cprint("3. Exit", "red")
    choice = input("Choose an option: ")
    return choice

# SAST Analysis
def analyze_firmware():
    cprint("Upload the firmware file for analysis.", "yellow")
    uploaded = files.upload()  # User uploads a firmware file
    firmware_path = list(uploaded.keys())[0]

    cprint(f"Analyzing firmware: {firmware_path}", "blue")

    # Check if it's a tar.gz file and extract accordingly
    if firmware_path.endswith(".tar") or firmware_path.endswith(".tar.gz"):
        extract_path = firmware_path + ".extracted"
        os.makedirs(extract_path, exist_ok=True)
        import tarfile
        with tarfile.open(firmware_path, "r:gz") as tar:
            tar.extractall(path=extract_path)

    elif firmware_path.endswith(".zip"):
        extract_path = firmware_path + ".extracted"
        os.makedirs(extract_path, exist_ok=True)
        import zipfile
        with zipfile.ZipFile(firmware_path, "r") as zip_ref:
            zip_ref.extractall(extract_path)

    else:
        cprint("Unsupported firmware format. Please provide a .tar, .zip, or .gz file.", "red")
        return []

    results = []

    # OWASP IoT Top 10 Tests for SAST
    cprint("Performing SAST checks for OWASP IoT Top 10...")

    # I1: Weak, Guessable, or Hardcoded Passwords
    cprint("Checking for hardcoded credentials...", "yellow")
    credentials_result = check_hardcoded_credentials(extract_path)
    results.append({"issue": "Weak, Guessable, or Hardcoded Passwords", "details": credentials_result})

    # I2: Insecure Network Services
    cprint("Scanning for insecure network services...", "yellow")
    network_services_result = scan_network_services(extract_path)
    results.append({"issue": "Insecure Network Services", "details": network_services_result})

    # I4: Lack of Secure Update Mechanism
    cprint("Checking firmware update mechanisms...", "yellow")
    update_mechanism_result = check_update_mechanisms(extract_path)
    results.append({"issue": "Lack of Secure Update Mechanism", "details": update_mechanism_result})

    return results





# DAST Analysis
def scan_network_device():
    target_ip = input("Enter the IP address of the IoT device: ")
    cprint(f"Scanning IP: {target_ip}...", "yellow")

    scanner = nmap.PortScanner()
    scanner.scan(hosts=target_ip, arguments='-sV')  # Service version detection

    results = []

    if target_ip in scanner.all_hosts():
        cprint(f"Scan Results for {target_ip}:", "green")
        for protocol in scanner[target_ip].all_protocols():
            ports = scanner[target_ip][protocol].keys()
            for port in ports:
                service = scanner[target_ip][protocol][port]['name']
                cve_details = fetch_cves(service)
                results.append({"port": port, "service": service, "cve_details": cve_details})
    else:
        cprint("No results found. Ensure the device is reachable.", "red")

    return results

# Helper Functions
def check_hardcoded_credentials(path):
    # Searching for common patterns in the firmware files (strings, passwords, keys, etc.)
    credentials = []
    files = os.listdir(path)

    for file in files:
        file_path = os.path.join(path, file)
        if os.path.isfile(file_path):
            output = subprocess.getoutput(f"strings {file_path}")
            matches = re.findall(r'[\w\-]+[:=][\w\-]+', output)  # Simple regex for patterns like user:pass
            credentials.extend(matches)

    if credentials:
        return f"Hardcoded credentials found: {', '.join(credentials)}"
    else:
        return "No hardcoded credentials detected."

def scan_network_services(path):
    # Add logic to check insecure network services (look for open ports, outdated services, etc.)
    insecure_services = []
    files = os.listdir(path)

    # Example of scanning for services using binwalk or extracted data
    for file in files:
        file_path = os.path.join(path, file)
        if os.path.isfile(file_path):
            if "telnet" in file.lower() or "ftp" in file.lower():
                insecure_services.append(file)

    if insecure_services:
        return f"Insecure services found: {', '.join(insecure_services)}"
    else:
        return "No insecure services detected."

def check_update_mechanisms(path):
    # Check if the firmware has secure update mechanisms (like HTTPS and cryptographic signatures)
    update_mechanism = []
    files = os.listdir(path)

    for file in files:
        file_path = os.path.join(path, file)
        if os.path.isfile(file_path):
            if "https" in open(file_path).read():
                update_mechanism.append("HTTPS found in update mechanism")
            if "signature" in open(file_path).read():
                update_mechanism.append("Cryptographic signature found")

    if update_mechanism:
        return f"Secure update mechanisms found: {', '.join(update_mechanism)}"
    else:
        return "No secure update mechanisms detected."

def fetch_cves(keyword):
    url = f"https://services.nvd.nist.gov/rest/json/cves/1.0?keyword={keyword}&resultsPerPage=3"
    response = requests.get(url)
    if response.status_code == 200:
        cves = response.json().get('result', {}).get('CVE_Items', [])
        return [{"id": cve['cve']['CVE_data_meta']['ID'], "description": cve['cve']['description']['description_data'][0]['value']} for cve in cves]
    else:
        return []

# Report Generation
def generate_report(data, format="pdf"):
    if format == "pdf":
        pdf = FPDF()
        pdf.set_auto_page_break(auto=True, margin=15)
        pdf.add_page()
        pdf.set_font("Arial", size=12)
        pdf.set_text_color(50, 50, 50)

        pdf.set_font("Arial", style='B', size=16)
        pdf.cell(200, 10, txt=TOOL_NAME, ln=True, align='C')

        pdf.set_font("Arial", size=12)
        pdf.ln(10)

        for item in data:
            pdf.set_font("Arial", style='B', size=14)
            pdf.cell(0, 10, txt=item['issue'], ln=True)
            pdf.set_font("Arial", size=12)
            pdf.multi_cell(0, 10, txt=item['details'])
            pdf.ln(5)

        pdf_file = "IoT_Security_Report.pdf"
        pdf.output(pdf_file)
        files.download(pdf_file)

    elif format == "json":
        json_file = "IoT_Security_Report.json"
        with open(json_file, "w") as f:
            json.dump(data, f, indent=4)
        files.download(json_file)

    elif format == "excel":
        df = pd.DataFrame(data)
        excel_file = "IoT_Security_Report.xlsx"
        df.to_excel(excel_file, index=False)
        files.download(excel_file)

def main():
    collected_results = []

    while True:
        choice = display_menu()
        if choice == "1":
            results = analyze_firmware()
            collected_results.extend(results)
            if results:
                cprint("SAST analysis complete. Would you like to generate a report? (yes/no)", "cyan")
                generate = input().strip().lower()
                if generate == "yes":
                    report_format = input("Choose report format (pdf/json/excel): ").strip().lower()
                    generate_report(results, format=report_format)
                cprint("Would you like to continue with DAST analysis? (yes/no)", "cyan")
                continue_dast = input().strip().lower()
                if continue_dast == "yes":
                    dast_results = scan_network_device()
                    collected_results.extend(dast_results)
                    if dast_results:
                        cprint("DAST analysis complete. Would you like to generate a report? (yes/no)", "cyan")
                        generate = input().strip().lower()
                        if generate == "yes":
                            report_format = input("Choose report format (pdf/json/excel): ").strip().lower()
                            generate_report(dast_results, format=report_format)
        elif choice == "2":
            dast_results = scan_network_device()
            collected_results.extend(dast_results)
            if dast_results:
                cprint("DAST analysis complete. Would you like to generate a report? (yes/no)", "cyan")
                generate = input().strip().lower()
                if generate == "yes":
                    report_format = input("Choose report format (pdf/json/excel): ").strip().lower()
                    generate_report(dast_results, format=report_format)
        elif choice == "3":
            cprint("Exiting the tool. Goodbye!", "red")
            break
        else:
            cprint("Invalid choice. Please try again.", "red")

main()


 ___    _____ ____                           ____                  
|_ _|__|_   _/ ___|  ___  ___ _   _ _ __ ___/ ___|  ___ __ _ _ __  
 | |/ _ \| | \___ \ / _ \/ __| | | | '__/ _ \___ \ / __/ _` | '_ \ 
 | | (_) | |  ___) |  __/ (__| |_| | | |  __/___) | (_| (_| | | | |
|___\___/|_| |____/ \___|\___|\__,_|_|  \___|____/ \___\__,_|_| |_|
                                                                   

1. Perform SAST Analysis (Firmware Scan)
2. Perform DAST Analysis (Network/Device Scan)
3. Exit
Choose an option: 1
Upload the firmware file for analysis.


Saving kit-contents.jpg to kit-contents.jpg
Saving LICENSE to LICENSE
Saving README.md to README.md
Analyzing firmware: kit-contents.jpg
Unsupported firmware format. Please provide a .tar, .zip, or .gz file.
 ___    _____ ____                           ____                  
|_ _|__|_   _/ ___|  ___  ___ _   _ _ __ ___/ ___|  ___ __ _ _ __  
 | |/ _ \| | \___ \ / _ \/ __| | | | '__/ _ \___ \ / __/ _` | '_ \ 
 | | (_) | |  ___) |  __/ (__| |_| | | |  __/___) | (_| (_| | | | |
|___\___/|_| |____/ \___|\___|\__,_|_|  \___|____/ \___\__,_|_| |_|
                                                                   

1. Perform SAST Analysis (Firmware Scan)
2. Perform DAST Analysis (Network/Device Scan)
3. Exit
Choose an option: 1
Upload the firmware file for analysis.


Saving DVID-master.zip to DVID-master (4).zip
Analyzing firmware: DVID-master (4).zip
Performing SAST checks for OWASP IoT Top 10...
Checking for hardcoded credentials...
Scanning for insecure network services...
Checking firmware update mechanisms...
SAST analysis complete. Would you like to generate a report? (yes/no)
yes
Choose report format (pdf/json/excel): excel


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Would you like to continue with DAST analysis? (yes/no)
no
 ___    _____ ____                           ____                  
|_ _|__|_   _/ ___|  ___  ___ _   _ _ __ ___/ ___|  ___ __ _ _ __  
 | |/ _ \| | \___ \ / _ \/ __| | | | '__/ _ \___ \ / __/ _` | '_ \ 
 | | (_) | |  ___) |  __/ (__| |_| | | |  __/___) | (_| (_| | | | |
|___\___/|_| |____/ \___|\___|\__,_|_|  \___|____/ \___\__,_|_| |_|
                                                                   

1. Perform SAST Analysis (Firmware Scan)
2. Perform DAST Analysis (Network/Device Scan)
3. Exit
Choose an option: 3
Exiting the tool. Goodbye!
